In [10]:
import cv2
import numpy as np

In [11]:
"""  
     Cette fonction permet de charger les poids et le fichier de configuration YoloV3 
     à l'aide du module dnn d'OpenCV.
     Le fichier coco.names contient les noms des différents objets que notre modèle 
     a été entraîné à identifier. Nous les stockons dans une liste appelée classes . Maintenant, 
     pour exécuter une passe en avant en utilisant le module cv2.dnn , nous devons passer les noms 
     des couches pour lesquelles la sortie doit être calculée. net.getUnconnectedOutLayers () retourne 
     les indices des couches de sortie du réseau.
"""

def load_yolo():
    net = cv2.dnn.readNet('yolov3-tiny.weights','yolov3-tiny.cfg')
    classes = []
    with open('coco.names','r') as f:
        classes = [line.strip() for line in f.readlines()]
    output_layers_names = net.getUnconnectedOutLayersNames()
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    return net, classes, colors, output_layers_names

In [12]:
""" 
    Pour accepter les fichiers image, nous aurons besoin d'une autre fonction appelée load_image () 
    qui acceptera un chemin d'image comme paramètre, lira l'image et la retournera.
"""
def load_image(img_path):
    img = cv2.imread(img_path)
    height, width, _ = img.shape
    return img, height, width

In [13]:
"""
    Pour prédire correctement les objets avec des réseaux de neurones profonds, nous devons prétraiter
    nos données et le module cv2.dnn nous fournit deux fonctions à cet effet: blobFromImage et blobFromImages. 
    Ces fonctions effectuent la mise à l'échelle, la soustraction de la moyenne et l'échange de canaux, ce qui est 
    facultatif. Nous utiliserons blobFromImage dans une fonction appelée detect_objects () qui accepte 
    les images / images du flux vidéo ou webcam, le modèle et les couches de sortie comme paramètres.
    Comme vous pouvez le voir dans l'extrait de code ci-dessus, nous avons utilisé le facteur d' échelle de 1/255,
    Par conséquent, nous mettons à l'échelle les pixels de l'image dans la plage de 0 à 1. Il n'y a pas besoin de 
    soustraction moyenne et c'est pourquoi nous la définissons sur la valeur mean [0, 0, 0] .
    La fonction forward () du module cv2.dnn renvoie une liste imbriquée contenant des informations sur tous 
    les objets détectés qui comprend les coordonnées x et y du centre de l'objet détecté, la hauteur et la largeur 
    de la boîte englobante, la confiance et les scores pour tous les classes d'objets répertoriés dans 
    coco.names. La classe avec le score le plus élevé est considérée comme la classe prédite.
"""

def detect_objects(img, net, outputLayers):
    blob = cv2.dnn.blobFromImage(img, scalefactor=1/255, size=(416, 416), mean=(0,0,0), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(outputLayers)
    return blob, layerOutputs


In [14]:
""" 
    Dans la fonction get_box_dimensions () , une liste appelée scores est créée qui stocke la confiance 
    correspondant à chaque objet. Nous identifions ensuite l'indice de classe avec le plus haut niveau de 
    confiance / score en utilisant np.argmax () . Nous pouvons obtenir le nom de la classe correspondant à l'index 
    à partir de la liste des classes que nous avons créée dans load_yolo () .
    J'ai sélectionné toutes les boîtes englobantes prévues avec une confiance de plus de 50%. Vous pouvez jouer 
    avec cette valeur.
"""

def get_box_dimensions(layerOutputs, height, width):
    boxes = []
    confidences = []
    class_ids = []
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)
                x = int(center_x - w/2)
                y = int(center_y - h/2)
                boxes.append([x, y, w, h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)
    return boxes, confidences, class_ids


In [15]:
""" 
    Maintenant que nous avons les sommets de la boîte englobante prédite et class_id (index de la classe d'objets 
    prédite), nous devons dessiner la boîte englobante et lui ajouter une étiquette d'objet. Nous ferons cela avec
    l'aide de la fonction draw_labels () .
    cv2.dnn.NMSBoxes () permet d'ajouter, dessiner la boîte englobante et y ajouter une étiquette, Elle permet 
    aussi de remédier à la situation que certains objets ont été détectés plusieurs fois et que nous avons plus 
    d'un cadre de délimitation pour un objet. Nous transmettons la valeur seuil de confiance et la valeur 
    seuil NMS (Suppression non-Maximum) comme paramètres pour sélectionner une boîte englobante. Dans la plage 
    de 0 à 1, nous devons sélectionner une valeur intermédiaire comme 0,4 ou 0,5 pour nous assurer 
    que nous détectons les objets qui se chevauchent mais que nous ne finissons pas par obtenir plusieurs cadres 
    de délimitation pour le même objet.
"""

def draw_labels(boxes, confidences, colors, class_ids, classes, img):
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i],2))
            color = colors[i]
            img = cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
            img = cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, color, 2)
    cv2.imshow('Image',img)
    

In [30]:
""" 
    Cette fonction image_detect () permet de détecter les objets dans le fichier image spécifié par leur chemin.
"""

def image_detect(img_path):
    model, classes, colors, output_layers_names = load_yolo()
    image, height, width = load_image(img_path)
    blob, layerOutputs = detect_objects(image, model, output_layers_names)
    boxes, confidences, class_ids = get_box_dimensions(layerOutputs, height, width)
    draw_labels(boxes, confidences, colors, class_ids, classes, image)
    while True:
        if cv2.waitKey(25) & 0xFF == ord('q'): # Enter q to quit
            cv2.destroyAllWindows() # détruire toutes les fenêtres cv2 affichées
            break
    cv2.destroyAllWindows()

In [31]:
image_detect('bicycle.jpg')

In [20]:
""" 
    De même, pour l'entrée webcam, nous avons crée la fonction start_webcam () pour le 
    real-time object detection. Cette fonction retourne l'instance de l'objet de capture d'image avec webcam
"""

def start_webcam():
    cap = cv2.VideoCapture(0)
    return cap

In [21]:
"""
    Cette fonction permet la détection d'objets en temps réel à l'aide de la webcam du PC
"""

def webcam_detect():
    model, classes, colors, output_layers = load_yolo()
    cap = start_webcam()
    if not cap.isOpened():
        raise IOError("We cannot open webcam !!!")
    while True:
        ret , frame = cap.read()
        height, width, channels = frame.shape
        blob, outputs = detect_objects(frame, model, output_layers)
        boxes, confs, class_ids = get_box_dimensions(outputs, height, width)
        draw_labels(boxes, confs, colors, class_ids, classes, frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    cap.release()
    cv2.destroyAllWindows()

In [32]:
"""
    De même, pour l'entrée vidéo, nous avons crée la fonction start_video () pour la détection d'objets sur un
    fichier vidéo spécifié par leur chemin.
"""

def start_video(video_path):
    model, classes, colors, output_layers = load_yolo()
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, frame = cap.read()
        height, width, channels = frame.shape
        blob, outputs = detect_objects(frame, model, output_layers)
        boxes, confs, class_ids = get_box_dimensions(outputs, height, width)
        draw_labels(boxes, confs, colors, class_ids, classes, frame)
        if cv2.waitKey(25) & 0xFF == ord('q'): # Enter q to quit
            cv2.destroyAllWindows() # détruire toutes les fenêtres cv2 affichées
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
webcam_detect()

In [35]:
start_video('pedestrians.mp4')